In [2]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

Matplotlib is building the font cache; this may take a moment.


#### create input file

In [2]:
input_df = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/business_nfcra_100k_07102019.csv", 
                        verbose = False, low_memory = False, dtype = str)
print(input_df.shape)

(100123, 117)


In [3]:
input_df = input_df.drop_duplicates(subset = ['CompanyName', 'AlternateCompanyName', 'StreetAddressLine1', 'StreetAddressLine2', 'City1', 
                                              'State1', 'Zip1', 'BusinessPhone', 'TaxIdNumber'], ignore_index = True)
print(input_df.shape)

(99205, 117)


In [4]:
input_df["BusinessStreetAddress"] = np.where(input_df.StreetAddressLine1.isnull(), "", input_df.StreetAddressLine1) + np.where(input_df.StreetAddressLine1.isnull() | input_df.StreetAddressLine2.isnull(), "", input_df.StreetAddressLine2)
input_df["BusinessStreetAddress"] = np.where(input_df["BusinessStreetAddress"] == "", None, input_df["BusinessStreetAddress"])
input_df["BusinessStreetAddress"].isnull().sum()

201

In [5]:
input_df["Rep1_StreetAddress"] = np.where(input_df.Rep1StreetAddressLine1.isnull(), "", input_df.Rep1StreetAddressLine1) + np.where(input_df.Rep1StreetAddressLine1.isnull() | input_df.Rep1StreetAddressLine2.isnull(), "", input_df.Rep1StreetAddressLine2)
input_df["Rep1_StreetAddress"] = np.where(input_df["Rep1_StreetAddress"] == "", None, input_df["Rep1_StreetAddress"])
input_df["Rep1_StreetAddress"].isnull().sum()

34419

In [6]:
input_df["Rep2_StreetAddress"] = np.where(input_df.Rep2StreetAddressLine1.isnull(), "", input_df.Rep2StreetAddressLine1) + np.where(input_df.Rep2StreetAddressLine1.isnull() | input_df.Rep2StreetAddressLine2.isnull(), "", input_df.Rep2StreetAddressLine2)
input_df["Rep2_StreetAddress"] = np.where(input_df["Rep2_StreetAddress"] == "", None, input_df["Rep2_StreetAddress"])
input_df["Rep2_StreetAddress"].isnull().sum()

89317

In [7]:
input_df["Rep3_StreetAddress"] = np.where(input_df.Rep3StreetAddressLine1.isnull(), "", input_df.Rep3StreetAddressLine1) + np.where(input_df.Rep3StreetAddressLine1.isnull() | input_df.Rep3StreetAddressLine2.isnull(), "", input_df.Rep3StreetAddressLine2)
input_df["Rep3_StreetAddress"] = np.where(input_df["Rep3_StreetAddress"] == "", None, input_df["Rep3_StreetAddress"])
input_df["Rep3_StreetAddress"].isnull().sum()

90633

In [8]:
input_df["sufficient_Business"] = np.where(input_df["CompanyName"].isnull() | ((input_df["BusinessStreetAddress"].isnull() | input_df["City1"].isnull() | input_df["State1"].isnull()) &
                            (input_df["BusinessStreetAddress"].isnull() | input_df["Zip1"].isnull())), 0, 1)

input_df["sufficient_input_rep1"] = np.where((input_df["Rep1FirstName"].notnull() & input_df["Rep1LastName"].notnull() & input_df["Rep1_StreetAddress"].notnull() & input_df["Rep1Zip"].notnull()) | \
             (input_df["Rep1FirstName"].notnull() & input_df["Rep1LastName"].notnull() & input_df["Rep1_StreetAddress"].notnull() & input_df["Rep1City"].notnull() & input_df["Rep1State"].notnull()) | \
             (input_df["Rep1FirstName"].notnull() & input_df["Rep1LastName"].notnull() & input_df["Rep1SSN"].notnull()) | \
             (input_df["Rep1FirstName"].notnull() & input_df["Rep1LastName"].notnull() & input_df["Rep1DOB"].notnull()), 1, 0)

input_df["sufficient_input_rep2"] = np.where((input_df["Rep2FirstName"].notnull() & input_df["Rep2LastName"].notnull() & input_df["Rep2_StreetAddress"].notnull() & input_df["Rep2Zip"].notnull()) | \
             (input_df["Rep2FirstName"].notnull() & input_df["Rep2LastName"].notnull() & input_df["Rep2_StreetAddress"].notnull() & input_df["Rep2City"].notnull() & input_df["Rep2State"].notnull()) | \
             (input_df["Rep2FirstName"].notnull() & input_df["Rep2LastName"].notnull() & input_df["Rep2SSN"].notnull()) | \
             (input_df["Rep2FirstName"].notnull() & input_df["Rep2LastName"].notnull() & input_df["Rep2DOB"].notnull()), 1, 0)

input_df["sufficient_input_rep3"] = np.where((input_df["Rep3FirstName"].notnull() & input_df["Rep3LastName"].notnull() & input_df["Rep3_StreetAddress"].notnull() & input_df["Rep3Zip"].notnull()) | \
             (input_df["Rep3FirstName"].notnull() & input_df["Rep3LastName"].notnull() & input_df["Rep3_StreetAddress"].notnull() & input_df["Rep3City"].notnull() & input_df["Rep3State"].notnull()) | \
             (input_df["Rep3FirstName"].notnull() & input_df["Rep3LastName"].notnull() & input_df["Rep3SSN"].notnull()) | \
             (input_df["Rep3FirstName"].notnull() & input_df["Rep3LastName"].notnull() & input_df["Rep3DOB"].notnull()), 1, 0)

In [9]:
freq(input_df["sufficient_Business"], input_df["sufficient_input_rep1"], input_df["sufficient_input_rep2"], input_df["sufficient_input_rep3"], cross = False, observed = True)

Count  \
sufficient_Business sufficient_input_rep1 sufficient_input_rep2 sufficient_input_rep3            
1                   0                     0                     0                      33309.0   
                                                                1                          4.0   
                                          1                     0                          1.0   
                    1                     0                     0                      55793.0   
                                                                1                         10.0   
                                          1                     0                       1322.0   
                                                                1                       8556.0   
0                   0                     0                     0                        131.0   
                    1                     0                     0                         71.0   
                                          1                     1                          8.0   

                                                                                            Pct  \
sufficient_Business sufficient_input_rep1 sufficient_input_rep2 sufficient_input_rep3             
1                   0                     0                     0                      0.335759   
                                                                1                      0.000040   
                                          1                     0                      0.000010   
                    1                     0                     0                      0.562401   
                                                                1                      0.000101   
                                          1                     0                      0.013326   
                                                                1                      0.086246   
0                   0                     0                     0                      0.001320   
                    1                     0                     0                      0.000716   
                                          1                     1                      0.000081   

                                                                                       Cuml Count  \
sufficient_Business sufficient_input_rep1 sufficient_input_rep2 sufficient_input_rep3               
1                   0                     0                     0                         33309.0   
                                                                1                         33313.0   
                                          1                     0                         33314.0   
                    1                     0                     0                         89107.0   
                                                                1                         89117.0   
                                          1                     0                         90439.0   
                                                                1                         98995.0   
0                   0                     0                     0                         99126.0   
                    1                     0                     0                         99197.0   
                                          1                     1                         99205.0   

                                                                                       Cuml Pct  
sufficient_Business sufficient_input_rep1 sufficient_input_rep2 sufficient_input_rep3            
1                   0                     0                     0                      0.335759  
                                                                1                      0.335800  
                                          1                     0                      0.335810  
                    

In [10]:
sample1 = input_df[(input_df["sufficient_Business"] ==1) & (input_df["sufficient_input_rep1"] ==1) & 
                   (input_df["sufficient_input_rep2"]==1) & (input_df["sufficient_input_rep3"] ==1)].sample(n = 7_000, replace = False, random_state = 1234)
sample1["group"] = "3rep"
sample2 = input_df[(input_df["sufficient_Business"] ==1) & (input_df["sufficient_input_rep1"] ==1) & 
                   (input_df["sufficient_input_rep2"]==1) & (input_df["sufficient_input_rep3"] ==0)].sample(n = 1_000, replace = False, random_state = 1234)
sample2["group"] = "2rep"
sample3 = input_df[(input_df["sufficient_Business"] ==1) & (input_df["sufficient_input_rep1"] ==1) & 
                   (input_df["sufficient_input_rep2"]==0) & (input_df["sufficient_input_rep3"] ==0)].sample(n = 1_000, replace = False, random_state = 1234)
sample3["group"] = "1rep"
sample4 = input_df[(input_df["sufficient_Business"] ==1) & (input_df["sufficient_input_rep1"] ==0) & 
                   (input_df["sufficient_input_rep2"]==0) & (input_df["sufficient_input_rep3"] ==0)].sample(n = 1_000, replace = False, random_state = 1234)
sample4["group"] = "0rep"

sample = pd.concat([sample1, sample2, sample3, sample4], ignore_index = True)
freq(sample.group)

,Count,Pct,Cuml Count,Cuml Pct
group,,,,
0rep,1000.0,0.1,1000.0,0.1
1rep,1000.0,0.1,2000.0,0.2
2rep,1000.0,0.1,3000.0,0.3
3rep,7000.0,0.7,10000.0,1.0


In [12]:
sample.AccountNumber.is_unique

True

In [13]:
sample.index

RangeIndex(start=0, stop=10000, step=1)

In [14]:
sample["unique_id"] = sample.index + 1

In [15]:
sample.to_adls('Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/input_sample_DS.parquet')

In [16]:
print(sample.shape)
sample["AccountNumber"] = sample["unique_id"]
sample = sample.drop(columns = ['BusinessStreetAddress', 'Rep1_StreetAddress', 'Rep2_StreetAddress', 'Rep3_StreetAddress', 'sufficient_Business', 'sufficient_input_rep1', 'sufficient_input_rep2',
                                 'sufficient_input_rep3', "unique_id"])
print(sample.shape)

(10000, 127)
(10000, 118)


In [18]:
sample = sample.drop(columns = ["group"])
sample.to_adls('Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/air231_input_sample_ECL.csv', index = False)

#### compare

In [10]:
baseline = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_small_business_busshellv31_air231_results_with1rep_new2w20230821-094133.csv", 
                        verbose = False, low_memory = False, encoding = 'iso-8859-1')
after = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_small_business_busshellv31_air231_results_with3rep_new2w20230821-102310.csv", 
                     verbose = False, low_memory = False, encoding = 'iso-8859-1')

In [12]:
baseline.shape

(10001, 2304)

In [3]:
# baseline = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_small_business_busshellv31_air231_results_with1rep_new2w20230821-094133.csv", 
#                         verbose = False, low_memory = False, encoding = 'iso-8859-1')
# after = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_small_business_busshellv31_air231_results_with3rep_new2w20230821-102310.csv", 
#                      verbose = False, low_memory = False, encoding = 'iso-8859-1')

baseline = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_data/1004_business_shell_v31_air231_results_with1rep_w20230928-162407_busshell.csv", 
                        verbose = False, low_memory = False, encoding = 'iso-8859-1')
after = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_data/1004_business_shell_v31_air231_results_with3rep_w20230928-103755_busshell.csv", 
                     verbose = False, low_memory = False, encoding = 'iso-8859-1')
print(baseline.shape)
print(after.shape)

(10001, 3966)
(10001, 3966)


In [6]:
pd.set_option('display.max_columns', None)
baseline.head()

ultidweight  ultidscore  orgidweight  orgidscore  seleidweight  \
0          127         100          127         100           127   
1            0           0            0           0             0   
2           78         100           78         100            78   
3            0           0            0           0             0   
4          119         100          119         100           119   

   seleidscore  proxidweight  proxidscore  powidweight  powidscore  \
0          100           127          100          127         100   
1            0             0            0            0           0   
2          100            78          100           78         100   
3            0             0            0            0           0   
4          100            90          100           90         100   

   input_echo.seq input_echo.acctno  input_echo.historydate  \
0               0              6095                  999999   
1               0              6151                  999999   
2               0              9585                  999999   
3               0              6134                  999999   
4               0              9664                  999999   

   input_echo.historydatetime  input_echo.powid  input_echo.proxid  \
0                      999999                 0                  0   
1                      999999                 0                  0   
2                      999999                 0                  0   
3                      999999                 0                  0   
4                      999999                 0                  0   

   input_echo.seleid  input_echo.orgid  input_echo.ultid  input_echo.resultid  \
0                  0                 0                 0                    0   
1                  0                 0                 0                    0   
2                  0                 0                 0                    0   
3                  0                 0                 0                    0   
4                  0                 0                 0                    0   

   input_echo.resultidweight  input_echo.resultidscore  input_echo.resorgid  \
0                          0                         0                    0   
1                          0                         0                    0   
2                          0                         0                    0   
3                          0                         0                    0   
4                          0                         0                    0   

   input_echo.resorgidweight  input_echo.resorgidscore  input_echo.resseleid  \
0                          0                         0                     0   
1                          0                         0                     0   
2                          0                         0                     0   
3                          0                         0                     0   
4                          0                         0                     0   

   input_echo.resseleidweight  input_echo.resseleidscore  \
0                           0                          0   
1                           0                          0   
2                           0                          0   
3                           0                          0   
4                           0                          0   

   input_echo.overallweight  input_echo.seleidchangeflag  \
0                         0                            0   
1                         0                            0   
2                         0                            0   
3                         0                            0   
4                         0                            0   

   input_echo.seleidchangecode    input_echo.companyname  \
0                            0           GORDON DESSERTS   
1                            0         SOCALIENTERPRISES   
2                     

In [13]:
baseline = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_business_shell_v31_air231_results_with1rep_new2_w20230821-105550_sas_layout_busshell.csv", 
                        verbose = False, low_memory = False, encoding = 'iso-8859-1')
after = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/new_business_shell_v31_air231_results_with3rep_new2_w20230821-110346_sas_layout_busshell.csv", 
                     verbose = False, low_memory = False, encoding = 'iso-8859-1')
print(baseline.shape)
print(after.shape)

(10001, 3115)
(10001, 3115)


In [15]:
baseline.head(3)

account  seq     id_powid    id_proxid    id_seleid     id_orgid  \
0    6205    0     27826869     27826869     27826869     27826869   
1    6204    0  64841960148  64841960148  64841960148  64841960148   
2    6211    0  85536216276  85536216276  44593738190  44593738190   

      id_ultid  id_seleid_change_flag  id_seleid_change_code  id_weight  \
0     27826869                      0                      0         90   
1  64841960148                      0                      0        151   
2  44593738190                      0                      0        141   

   id_score  id_category id_status  id_truebiz  history_date  \
0       100  6 DUAL CORE      GOLD           1        999999   
1       100  6 DUAL CORE      GOLD           1        999999   
2       100  6 DUAL CORE      GOLD           1        999999   

   history_datetime                 in_bus_name in_bus_alternatename  \
0            999999  CC K CLEANING SERVICES LLC                  NaN   
1            999999    EG TRAVEL AND TOURS  LLC                  NaN   
2            999999           SMM INTERNATIONAL                  NaN   

  in_bus_streetaddress1  in_bus_streetaddress2 in_bus_city in_bus_state  \
0         1017 CRAIG ST                    NaN  MCKEESPORT           PA   
1       727 COCKATOO CT                    NaN   KISSIMMEE           FL   
2    315 ARBOR RIDGE LN                    NaN      CONROE           TX   

  in_bus_zip out_bus_prim_range out_bus_predir out_bus_prim_name  \
0      15132               1017            NaN             CRAIG   
1      34759                727            NaN          COCKATOO   
2      77384                315            NaN       ARBOR RIDGE   

  out_bus_addr_suffix out_bus_postdir out_bus_unit_desig out_bus_sec_range  \
0                  ST             NaN                NaN               NaN   
1                  CT             NaN                NaN               NaN   
2                  LN             NaN                NaN               NaN   

  out_bus_state  out_bus_zip5  out_bus_zip4  out_bus_lat  out_bus_long  \
0            PA       15132.0        1807.0    40.346680    -79.839149   
1            FL       34759.0        4510.0    28.107219    -81.458980   
2            TX       77384.0        3724.0    30.242169    -95.495979   

  out_bus_addr_type out_bus_addr_status  out_bus_county  out_bus_geo_block  \
0                 S                S800             3.0          5512003.0   
1                 S                S800           105.0           125032.0   
2                 S                S800           339.0          6905002.0   

  in_bus_phone10  in_bus_fax in_bus_ipaddr  in_bus_url in_bus_naics  \
0     4129156149         NaN           NaN         NaN          NaN   
1     4079948963         NaN           NaN         NaN          NaN   
2     8329285304         NaN           NaN         NaN          NaN   

  in_bus_sic in_rep1_first in_rep1_middle in_rep1_streetaddress1  \
0        NaN         CARLA              J          1017 CRAIG ST   
1        NaN        ESTHER            NaN        727 COCKATOO CT   
2        NaN   CHRISTOPHER            NaN     315 ARBOR RIDGE LN   

   in_rep1_streetaddress2 in_rep1_city in_rep1_state in_rep1_zip  \
0                     NaN  MC KEESPORT            PA       15132   
1                     NaN    KISSIMMEE            FL       34759   
2                     NaN       CONROE            TX       77384   

  out_rep1_prim_range out_rep1_predir out_rep1_prim_name out_rep1_addr_suffix  \
0                1017             NaN              CRAIG                   ST   
1                 727             NaN           COCKATOO                   CT   
2                 315             NaN        ARBOR RIDGE                   LN   

  out_rep1_postdir out_rep1_unit_desig out_rep1_sec_range out_rep1_state  \
0              NaN                 NaN                NaN             PA   
1              NaN                 NaN                NaN            

In [3]:
baseline.account.is_unique

True

In [4]:
after.account.is_unique

True

In [8]:
after.columns

Index(['ultidweight', 'ultidscore', 'orgidweight', 'orgidscore',
       'seleidweight', 'seleidscore', 'proxidweight', 'proxidscore',
       'powidweight', 'powidscore',
       ...
       'jpmc.jpmctotcredremmax', 'jpmc.jpmctotalcurcredlmt',
       'jpmc.jpmccurrentutilrevnocf', 'jpmc.jpmccurrentutilcardnocf',
       'jpmc.jpmcutilfin12mnocf', 'jpmc.jpmcutilrev3mnocf',
       'jpmc.jpmcsumvelocityavg11mrev', 'jpmc.jpmcsumvelocityavg48mrev',
       'jpmc.filteredsbfeaccountcount', 'errorcode'],
      dtype='object', length=3966)

In [9]:
from dsgtools.reporting import ImpactAnalysisReport
sbfe_attr = [x for x in baseline.columns if "sbfe" in x]
b2b = [x for x in baseline.columns if "b2b" in x]
rv_attr = [x for x in baseline.columns if x not in sbfe_attr]

exception_dict = dict()
for x in baseline.columns:
    if x in sbfe_attr:
        exception_dict[x] = [-99, -98, -97]
    if x in b2b:
        exception_dict[x] = [-99999, -99998, -99997]
    if x in rv_attr:
        exception_dict[x] = [-1]
        
cr = ImpactAnalysisReport(baseline, after, keys="input_echo.acctno",
           exceptions=exception_dict,
           dataframe_names = ("baseline", "after"))
cr.to_xlsx("./_temp/comparison.xlsx", overwrite= True)

In [40]:
temp_before = baseline[field].copy()
temp_after = after[field].copy()
sample = pd.read_adls('Analytics/RnD Projects/Product RnD/Business/_ARMBS_Tickets/2023/AIR_231 (Multi_rep)/input_sample_DS.parquet') #AccountNumber
sample.unique_id = sample.unique_id.astype(str)
temp_before = temp_before.merge(temp_after, on = "account")
temp_before = temp_before.merge(sample[["unique_id", "group"]], left_on = "account", right_on = "unique_id")
print(temp_before.shape)
temp_before = temp_before.fillna("")

(10000, 321)


In [72]:
col_2 = [x for x in temp_before.columns if ("_flag" in x) & ("rep2" in x) & ("in_rep2_dob" not in x)] ## rep dob is provided as 0 somehow
col_3 = [x for x in temp_before.columns if ("_flag" in x) & ("rep3" in x) & ("in_rep3_dob" not in x)]
print(len(col_2))
print(len(col_3))
temp_before['sum_diff_rep2']=temp_before[col_2]. sum(axis=1)
temp_before['sum_diff_rep3']=temp_before[col_3]. sum(axis=1)

78
78


In [73]:
fmt = make_format(cuts = [-np.inf, 0, np.inf])
freq("group", "sum_diff_rep3", df = temp_before, format = [None, fmt], cross = False, observed = True)

,,Count,Pct,Cuml Count,Cuml Pct
group,sum_diff_rep3,,,,
3rep,1+,7000.0,0.7,7000.0,0.7
1rep,<= 0,1000.0,0.1,8000.0,0.8
0rep,<= 0,1000.0,0.1,9000.0,0.9
2rep,<= 0,1000.0,0.1,10000.0,1.0


In [74]:
fmt = make_format(cuts = [-np.inf, 0, np.inf])
freq("group", "sum_diff_rep2", df = temp_before, format = [None, fmt], cross = False, observed = True)

Count     Pct  Cuml Count  Cuml Pct
group sum_diff_rep2                                      
3rep  1+             7000.0  0.7000      7000.0    0.7000
1rep  <= 0           1000.0  0.1000      8000.0    0.8000
0rep  <= 0            999.0  0.0999      8999.0    0.8999
      1+                1.0  0.0001      9000.0    0.9000
2rep  1+             1000.0  0.1000     10000.0    1.0000

In [79]:
sample[sample["unique_id"] == "9617"]

,AccountNumber,CompanyName,AlternateCompanyName,StreetAddressLine1,StreetAddressLine2,City1,State1,Zip1,BusinessPhone,TaxIdNumber,BusinessIPAddress,BusinessURL,BusinessEmailAddress,Rep1FirstName,Rep1MiddleName,Rep1LastName,Rep1NameSuffix,Rep1StreetAddressLine1,Rep1StreetAddressLine2,Rep1City,Rep1State,Rep1Zip,Rep1SSN,Rep1DOB,Rep1Age,Rep1DLNumber,Rep1DLState,Rep1HomePhone,Rep1EmailAddress,Rep1FormerLastName,Rep1LexID,ArchiveDate,PowID,ProxID,SeleID,OrgID,UltID,SIC_Code,NAIC_Code,Rep2FirstName,Rep2MiddleName,Rep2LastName,Rep2NameSuffix,Rep2StreetAddressLine1,Rep2StreetAddressLine2,Rep2City,Rep2State,Rep2Zip,Rep2SSN,Rep2DOB,Rep2Age,Rep2DLNumber,Rep2DLState,Rep2HomePhone,Rep2EmailAddress,Rep2FormerLastName,Rep2LexID,Rep3FirstName,Rep3MiddleName,Rep3LastName,Rep3NameSuffix,Rep3StreetAddressLine1,Rep3StreetAddressLine2,Rep3City,Rep3State,Rep3Zip,Rep3SSN,Rep3DOB,Rep3Age,Rep3DLNumber,Rep3DLState,Rep3HomePhone,Rep3EmailAddress,Rep3FormerLastName,Rep3LexID,Rep4FirstName,Rep4MiddleName,Rep4LastName,Rep4NameSuffix,Rep4StreetAddressLine1,Rep4StreetAddressLine2,Rep4City,Rep4State,Rep4Zip,Rep4SSN,Rep4DOB,Rep4Age,Rep4DLNumber,Rep4DLState,Rep4HomePhone,Rep4EmailAddress,Rep4FormerLastName,Rep4LexID,Rep5FirstName,Rep5MiddleName,Rep5LastName,Rep5NameSuffix,Rep5StreetAddressLine1,Rep5StreetAddressLine2,Rep5City,Rep5State,Rep5Zip,Rep5SSN,Rep5DOB,Rep5Age,Rep5DLNumber,Rep5DLState,Rep5HomePhone,Rep5EmailAddress,Rep5FormerLastName,Rep5LexID,ln_project_id,pf_fraud,pf_bad,pf_funded,pf_declined,pf_approved_not_funded,BusinessStreetAddress,Rep1_StreetAddress,Rep2_StreetAddress,Rep3_StreetAddress,sufficient_Business,sufficient_input_rep1,sufficient_input_rep2,sufficient_input_rep3,group,unique_id
9616,BIIDTransactionJune2018_50968041R1098689,"VENTURE.CO HOLDINGS, INC.",None,12 EAST 49TH ST 15TH FLOOR,None,NEW YORK,NY,10017,8447806797,None,None,None,None,JUSTIN,None,WICKS,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,20180629 200513,None,None,None,None,None,None,None,ANDREW,None,SZABO,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,BIID_Transaction_June2018,None,None,None,None,None,12 EAST 49TH ST 15TH FLOOR,None,None,None,1,0,0,0,0rep,9617


In [77]:
pd.set_option('display.max_columns', None)
temp_before.loc[(temp_before["group"] == "0rep") & (temp_before.sum_diff_rep2 >0), ["account", "sum_diff_rep2"] + col_2].head() 
## re2 input is there just insufficient
# temp_before.loc[(temp_before["group"] == "1rep") & (temp_before.sum_diff_rep2 >0), ["account", "in_rep2_dob_flag", "in_rep2_dob_x", "in_rep2_dob_y"]].head() #in_rep2_dob_flag

,account,sum_diff_rep2,in_rep2_first_flag,pop_rep2_last_flag,pop_rep2_first_flag,e2b_rep2_match_bus_in_first_flag,e2b_rep2_match_bus_in_last_flag,e2b_rep2_match_bus_in_full_flag,out_rep2_addr_status_flag,pop_rep2_zip_flag,pop_rep2_addr_flag,in_rep2_streetaddress1_flag,in_rep2_zip_flag,in_rep2_city_flag,pop_rep2_city_flag,out_rep2_state_flag,out_rep2_addr_type_flag,out_rep2_prim_name_flag,out_rep2_zip5_flag,out_rep2_lat_flag,out_rep2_long_flag,out_rep2_geo_block_flag,out_rep2_county_flag,out_rep2_prim_range_flag,pop_rep2_ssn_flag,lexid_rep2_flag,pop_rep2_state_flag,in_rep2_state_flag,out_rep2_zip4_flag,out_rep2_addr_suffix_flag,in_rep2_phone10_flag,pop_rep2_phone_flag,e2b_rep2_on_inquiry_flag,bus2exec_rep2_desc_flag,e2b_rep2_match_bus_in_addr_flag,bus2exec_rep2_flag,e2b_rep2_idsearch_name_flag,e2b_rep2_lexid_on_file_flag,e2b_rep2_name_on_file_flag,e2b_rep2_on_property_cnt_flag,e2b_rep2_on_utility_flag,e2b_rep2_match_bus_file_full_flag,e2b_rep2_paw_match_flag,e2b_rep2_match_bus_file_first_flag,e2b_rep2_match_bus_file_last_flag,e2b_rep2_idsearch_addr_flag,e2b_rep2_match_bus_file_addr_flag,e2b_rep2_consheader_addr_flag,e2b_rep2_addr_on_file_flag,e2b_rep2_bus_addr_own_cnt_flag,e2b_rep2_match_bus_file_fein_flag,e2b_rep2_busheader_ssn_flag,e2b_rep2_consheader_ssn_flag,e2b_rep2_idsearch_ssn_flag,e2b_rep2_ssn_on_file_flag,e2b_rep2_distance_addr_flag,e2b_rep2_busheader_phone_flag,e2b_rep2_phn_on_file_flag,e2b_rep2_idsearch_phone_flag,e2b_rep2_match_bus_file_phn_flag,e2b_rep2_consheader_phone_flag,bus2exec_rep2_title_flag,pop_rep2_dobyear_flag,pop_rep2_dob_flag,pop_rep2_dobday_flag,in_rep2_middle_flag,pop_rep2_middle_flag,out_rep2_predir_flag,e2b_rep2_distance_phone_flag,out_rep2_sec_range_flag,out_rep2_unit_desig_flag,out_rep2_postdir_flag,pop_rep2_dobmonth_flag,in_rep2_dlnumber_flag,pop_rep2_dlnumber_flag,in_rep2_dlstate_flag,pop_rep2_dlstate_flag,in_rep2_email_flag,pop_rep2_age_flag,in_rep2_age_flag
814,9617,17,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
freq("group", "e2b_rep1_consheader_addr_x", "e2b_rep1_consheader_addr_y", cross = False, observed = True, df = temp_before.loc[temp_before.e2b_rep1_consheader_addr_flag ==1])
# grab samples

,,,Count,Pct,Cuml Count,Cuml Pct
group,e2b_rep1_consheader_addr_x,e2b_rep1_consheader_addr_y,,,,
3rep,1,0,51.0,0.980769,51.0,0.980769
2rep,1,0,1.0,0.019231,52.0,1.000000


In [82]:
sample[sample.unique_id == "4245"]

,AccountNumber,CompanyName,AlternateCompanyName,StreetAddressLine1,StreetAddressLine2,City1,State1,Zip1,BusinessPhone,TaxIdNumber,BusinessIPAddress,BusinessURL,BusinessEmailAddress,Rep1FirstName,Rep1MiddleName,Rep1LastName,Rep1NameSuffix,Rep1StreetAddressLine1,Rep1StreetAddressLine2,Rep1City,Rep1State,Rep1Zip,Rep1SSN,Rep1DOB,Rep1Age,Rep1DLNumber,Rep1DLState,Rep1HomePhone,Rep1EmailAddress,Rep1FormerLastName,Rep1LexID,ArchiveDate,PowID,ProxID,SeleID,OrgID,UltID,SIC_Code,NAIC_Code,Rep2FirstName,Rep2MiddleName,Rep2LastName,Rep2NameSuffix,Rep2StreetAddressLine1,Rep2StreetAddressLine2,Rep2City,Rep2State,Rep2Zip,Rep2SSN,Rep2DOB,Rep2Age,Rep2DLNumber,Rep2DLState,Rep2HomePhone,Rep2EmailAddress,Rep2FormerLastName,Rep2LexID,Rep3FirstName,Rep3MiddleName,Rep3LastName,Rep3NameSuffix,Rep3StreetAddressLine1,Rep3StreetAddressLine2,Rep3City,Rep3State,Rep3Zip,Rep3SSN,Rep3DOB,Rep3Age,Rep3DLNumber,Rep3DLState,Rep3HomePhone,Rep3EmailAddress,Rep3FormerLastName,Rep3LexID,Rep4FirstName,Rep4MiddleName,Rep4LastName,Rep4NameSuffix,Rep4StreetAddressLine1,Rep4StreetAddressLine2,Rep4City,Rep4State,Rep4Zip,Rep4SSN,Rep4DOB,Rep4Age,Rep4DLNumber,Rep4DLState,Rep4HomePhone,Rep4EmailAddress,Rep4FormerLastName,Rep4LexID,Rep5FirstName,Rep5MiddleName,Rep5LastName,Rep5NameSuffix,Rep5StreetAddressLine1,Rep5StreetAddressLine2,Rep5City,Rep5State,Rep5Zip,Rep5SSN,Rep5DOB,Rep5Age,Rep5DLNumber,Rep5DLState,Rep5HomePhone,Rep5EmailAddress,Rep5FormerLastName,Rep5LexID,ln_project_id,pf_fraud,pf_bad,pf_funded,pf_declined,pf_approved_not_funded,BusinessStreetAddress,Rep1_StreetAddress,Rep2_StreetAddress,Rep3_StreetAddress,sufficient_Business,sufficient_input_rep1,sufficient_input_rep2,sufficient_input_rep3,group,unique_id
4244,cancapital7343_0021854610671006,"B-JAY, INC",Ben's Tires & Brakes,1140 S La Brea Ave,None,Inglewood,CA,90301,3106710418,264032772,None,None,None,Benjamin,None,Gonzalez,None,8915 DUDLEXT AVE,None,South Gate,CA,90280,545930141,1985-02-03,None,None,None,None,None,None,None,201501,None,None,None,None,None,None,None,Benjamin,None,Gonzalez,None,8915 DUDLEXT AVE,None,South Gate,CA,90280,545930141,1985-02-03,None,None,None,None,None,None,None,Benjamin,None,Gonzalez,None,8915 DUDLEXT AVE,None,South Gate,CA,90280,545930141,1985-02-03,None,None,None,None,None,None,None,Benjamin,None,Gonzalez,None,8915 DUDLEXT AVE,None,South Gate,CA,90280,545930141,1985-02-03,None,None,None,None,None,None,None,Benjamin,None,Gonzalez,None,8915 DUDLEXT AVE,None,South Gate,CA,90280,545930141,1985-02-03,None,None,None,None,None,None,None,can_capital_7343,0,None,1,None,0,1140 S La Brea Ave,8915 DUDLEXT AVE,8915 DUDLEXT AVE,8915 DUDLEXT AVE,1,1,1,1,3rep,4245


In [81]:
temp_before.loc[temp_before.e2b_rep1_consheader_addr_flag ==1, ["unique_id", "group", ]]

,unique_id,group
910,4245,3rep
1358,5579,3rep
1418,3484,3rep
1626,4059,3rep
2000,1463,3rep
2214,6318,3rep
2380,3266,3rep
2426,2184,3rep
2717,4769,3rep
3009,3524,3rep


In [41]:
temp_before['in_rep2_first_flag'] = np.where(temp_before.in_rep2_first_x != temp_before.in_rep2_first_y, 1, 0)
temp_before['pop_rep2_last_flag'] = np.where(temp_before.pop_rep2_last_x != temp_before.pop_rep2_last_y, 1, 0)
temp_before['pop_rep2_first_flag'] = np.where(temp_before.pop_rep2_first_x != temp_before.pop_rep2_first_y, 1, 0)
temp_before['e2b_rep2_match_bus_in_first_flag'] = np.where(temp_before.e2b_rep2_match_bus_in_first_x != temp_before.e2b_rep2_match_bus_in_first_y, 1, 0)
temp_before['e2b_rep2_match_bus_in_last_flag'] = np.where(temp_before.e2b_rep2_match_bus_in_last_x != temp_before.e2b_rep2_match_bus_in_last_y, 1, 0)
temp_before['e2b_rep2_match_bus_in_full_flag'] = np.where(temp_before.e2b_rep2_match_bus_in_full_x != temp_before.e2b_rep2_match_bus_in_full_y, 1, 0)
temp_before['out_rep2_addr_status_flag'] = np.where(temp_before.out_rep2_addr_status_x != temp_before.out_rep2_addr_status_y, 1, 0)
temp_before['pop_rep2_zip_flag'] = np.where(temp_before.pop_rep2_zip_x != temp_before.pop_rep2_zip_y, 1, 0)
temp_before['pop_rep2_addr_flag'] = np.where(temp_before.pop_rep2_addr_x != temp_before.pop_rep2_addr_y, 1, 0)
temp_before['in_rep2_streetaddress1_flag'] = np.where(temp_before.in_rep2_streetaddress1_x != temp_before.in_rep2_streetaddress1_y, 1, 0)
temp_before['in_rep2_zip_flag'] = np.where(temp_before.in_rep2_zip_x != temp_before.in_rep2_zip_y, 1, 0)
temp_before['in_rep2_city_flag'] = np.where(temp_before.in_rep2_city_x != temp_before.in_rep2_city_y, 1, 0)
temp_before['pop_rep2_city_flag'] = np.where(temp_before.pop_rep2_city_x != temp_before.pop_rep2_city_y, 1, 0)
temp_before['out_rep2_state_flag'] = np.where(temp_before.out_rep2_state_x != temp_before.out_rep2_state_y, 1, 0)
temp_before['out_rep2_addr_type_flag'] = np.where(temp_before.out_rep2_addr_type_x != temp_before.out_rep2_addr_type_y, 1, 0)
temp_before['out_rep2_prim_name_flag'] = np.where(temp_before.out_rep2_prim_name_x != temp_before.out_rep2_prim_name_y, 1, 0)
temp_before['out_rep2_zip5_flag'] = np.where(temp_before.out_rep2_zip5_x != temp_before.out_rep2_zip5_y, 1, 0)
temp_before['out_rep2_lat_flag'] = np.where(temp_before.out_rep2_lat_x != temp_before.out_rep2_lat_y, 1, 0)
temp_before['out_rep2_long_flag'] = np.where(temp_before.out_rep2_long_x != temp_before.out_rep2_long_y, 1, 0)
temp_before['out_rep2_geo_block_flag'] = np.where(temp_before.out_rep2_geo_block_x != temp_before.out_rep2_geo_block_y, 1, 0)
temp_before['out_rep2_county_flag'] = np.where(temp_before.out_rep2_county_x != temp_before.out_rep2_county_y, 1, 0)
temp_before['out_rep2_prim_range_flag'] = np.where(temp_before.out_rep2_prim_range_x != temp_before.out_rep2_prim_range_y, 1, 0)
temp_before['pop_rep2_ssn_flag'] = np.where(temp_before.pop_rep2_ssn_x != temp_before.pop_rep2_ssn_y, 1, 0)
temp_before['lexid_rep2_flag'] = np.where(temp_before.lexid_rep2_x != temp_before.lexid_rep2_y, 1, 0)
temp_before['pop_rep2_state_flag'] = np.where(temp_before.pop_rep2_state_x != temp_before.pop_rep2_state_y, 1, 0)
temp_before['in_rep2_state_flag'] = np.where(temp_before.in_rep2_state_x != temp_before.in_rep2_state_y, 1, 0)
temp_before['out_rep2_zip4_flag'] = np.where(temp_before.out_rep2_zip4_x != temp_before.out_rep2_zip4_y, 1, 0)
temp_before['out_rep2_addr_suffix_flag'] = np.where(temp_before.out_rep2_addr_suffix_x != temp_before.out_rep2_addr_suffix_y, 1, 0)
temp_before['in_rep2_phone10_flag'] = np.where(temp_before.in_rep2_phone10_x != temp_before.in_rep2_phone10_y, 1, 0)
temp_before['pop_rep2_phone_flag'] = np.where(temp_before.pop_rep2_phone_x != temp_before.pop_rep2_phone_y, 1, 0)
temp_before['e2b_rep2_on_inquiry_flag'] = np.where(temp_before.e2b_rep2_on_inquiry_x != temp_before.e2b_rep2_on_inquiry_y, 1, 0)
temp_before['bus2exec_rep2_desc_flag'] = np.where(temp_before.bus2exec_rep2_desc_x != temp_before.bus2exec_rep2_desc_y, 1, 0)
temp_before['e2b_rep2_match_bus_in_addr_flag'] = np.where(temp_before.e2b_rep2_match_bus_in_addr_x != temp_before.e2b_rep2_match_bus_in_addr_y, 1, 0)
temp_before['in_rep3_first_flag'] = np.where(temp_before.in_rep3_first_x != temp_before.in_rep3_first_y, 1, 0)
temp_before['e2b_rep3_match_bus_in_last_flag'] = np.where(temp_before.e2b_rep3_match_bus_in_last_x != temp_before.e2b_rep3_match_bus_in_last_y, 1, 0)
temp_before['pop_rep3_first_flag'] = np.where(temp_before.pop_rep3_first_x != temp_before.pop_rep3_first_y, 1, 0)
temp_before['pop_rep3_last_flag'] = np.where(temp_before.pop_rep3_last_x != temp_before.pop_rep3_last_y, 1, 0)
temp_before['e2b_rep3_match_bus_in_first_flag'] = np.where(temp_before.e2b_rep3_match_bus_in_first_x != temp_before.e2b_rep3_match_bus_in_first_y, 1, 0)
temp_before['e2b_rep3_match_bus_in_full_flag'] = np.where(temp_before.e2b_rep3_match_bus_in_full_x != temp_before.e2b_rep3_match_bus_in_full_y, 1, 0)
temp_before['out_rep3_addr_status_flag'] = np.where(temp_before.out_rep3_addr_status_x != temp_before.out_rep3_addr_status_y, 1, 0)
temp_before['in_rep3_streetaddress1_flag'] = np.where(temp_before.in_rep3_streetaddress1_x != temp_before.in_rep3_streetaddress1_y, 1, 0)
temp_before['pop_rep3_addr_flag'] = np.where(temp_before.pop_rep3_addr_x != temp_before.pop_rep3_addr_y, 1, 0)
temp_before['in_rep3_zip_flag'] = np.where(temp_before.in_rep3_zip_x != temp_before.in_rep3_zip_y, 1, 0)
temp_before['pop_rep3_zip_flag'] = np.where(temp_before.pop_rep3_zip_x != temp_before.pop_rep3_zip_y, 1, 0)
temp_before['in_rep3_city_flag'] = np.where(temp_before.in_rep3_city_x != temp_before.in_rep3_city_y, 1, 0)
temp_before['pop_rep3_city_flag'] = np.where(temp_before.pop_rep3_city_x != temp_before.pop_rep3_city_y, 1, 0)
temp_before['pop_rep3_state_flag'] = np.where(temp_before.pop_rep3_state_x != temp_before.pop_rep3_state_y, 1, 0)
temp_before['in_rep3_state_flag'] = np.where(temp_before.in_rep3_state_x != temp_before.in_rep3_state_y, 1, 0)
temp_before['out_rep3_state_flag'] = np.where(temp_before.out_rep3_state_x != temp_before.out_rep3_state_y, 1, 0)
temp_before['out_rep3_prim_name_flag'] = np.where(temp_before.out_rep3_prim_name_x != temp_before.out_rep3_prim_name_y, 1, 0)
temp_before['out_rep3_addr_type_flag'] = np.where(temp_before.out_rep3_addr_type_x != temp_before.out_rep3_addr_type_y, 1, 0)
temp_before['out_rep3_zip5_flag'] = np.where(temp_before.out_rep3_zip5_x != temp_before.out_rep3_zip5_y, 1, 0)
temp_before['out_rep3_geo_block_flag'] = np.where(temp_before.out_rep3_geo_block_x != temp_before.out_rep3_geo_block_y, 1, 0)
temp_before['out_rep3_long_flag'] = np.where(temp_before.out_rep3_long_x != temp_before.out_rep3_long_y, 1, 0)
temp_before['out_rep3_lat_flag'] = np.where(temp_before.out_rep3_lat_x != temp_before.out_rep3_lat_y, 1, 0)
temp_before['out_rep3_county_flag'] = np.where(temp_before.out_rep3_county_x != temp_before.out_rep3_county_y, 1, 0)
temp_before['out_rep3_prim_range_flag'] = np.where(temp_before.out_rep3_prim_range_x != temp_before.out_rep3_prim_range_y, 1, 0)
temp_before['lexid_rep3_flag'] = np.where(temp_before.lexid_rep3_x != temp_before.lexid_rep3_y, 1, 0)
temp_before['in_rep3_phone10_flag'] = np.where(temp_before.in_rep3_phone10_x != temp_before.in_rep3_phone10_y, 1, 0)
temp_before['pop_rep3_phone_flag'] = np.where(temp_before.pop_rep3_phone_x != temp_before.pop_rep3_phone_y, 1, 0)
temp_before['bus2exec_rep2_flag'] = np.where(temp_before.bus2exec_rep2_x != temp_before.bus2exec_rep2_y, 1, 0)
temp_before['out_rep3_zip4_flag'] = np.where(temp_before.out_rep3_zip4_x != temp_before.out_rep3_zip4_y, 1, 0)
temp_before['e2b_rep2_idsearch_name_flag'] = np.where(temp_before.e2b_rep2_idsearch_name_x != temp_before.e2b_rep2_idsearch_name_y, 1, 0)
temp_before['e2b_rep2_lexid_on_file_flag'] = np.where(temp_before.e2b_rep2_lexid_on_file_x != temp_before.e2b_rep2_lexid_on_file_y, 1, 0)
temp_before['e2b_rep2_name_on_file_flag'] = np.where(temp_before.e2b_rep2_name_on_file_x != temp_before.e2b_rep2_name_on_file_y, 1, 0)
temp_before['e2b_rep2_on_property_cnt_flag'] = np.where(temp_before.e2b_rep2_on_property_cnt_x != temp_before.e2b_rep2_on_property_cnt_y, 1, 0)
temp_before['e2b_rep2_on_utility_flag'] = np.where(temp_before.e2b_rep2_on_utility_x != temp_before.e2b_rep2_on_utility_y, 1, 0)
temp_before['e2b_rep2_match_bus_file_full_flag'] = np.where(temp_before.e2b_rep2_match_bus_file_full_x != temp_before.e2b_rep2_match_bus_file_full_y, 1, 0)
temp_before['e2b_rep2_paw_match_flag'] = np.where(temp_before.e2b_rep2_paw_match_x != temp_before.e2b_rep2_paw_match_y, 1, 0)
temp_before['e2b_rep2_match_bus_file_first_flag'] = np.where(temp_before.e2b_rep2_match_bus_file_first_x != temp_before.e2b_rep2_match_bus_file_first_y, 1, 0)
temp_before['e2b_rep2_match_bus_file_last_flag'] = np.where(temp_before.e2b_rep2_match_bus_file_last_x != temp_before.e2b_rep2_match_bus_file_last_y, 1, 0)
temp_before['e2b_rep2_idsearch_addr_flag'] = np.where(temp_before.e2b_rep2_idsearch_addr_x != temp_before.e2b_rep2_idsearch_addr_y, 1, 0)
temp_before['e2b_rep2_match_bus_file_addr_flag'] = np.where(temp_before.e2b_rep2_match_bus_file_addr_x != temp_before.e2b_rep2_match_bus_file_addr_y, 1, 0)
temp_before['e2b_rep2_consheader_addr_flag'] = np.where(temp_before.e2b_rep2_consheader_addr_x != temp_before.e2b_rep2_consheader_addr_y, 1, 0)
temp_before['e2b_rep2_addr_on_file_flag'] = np.where(temp_before.e2b_rep2_addr_on_file_x != temp_before.e2b_rep2_addr_on_file_y, 1, 0)
temp_before['e2b_rep2_bus_addr_own_cnt_flag'] = np.where(temp_before.e2b_rep2_bus_addr_own_cnt_x != temp_before.e2b_rep2_bus_addr_own_cnt_y, 1, 0)
temp_before['e2b_rep2_match_bus_file_fein_flag'] = np.where(temp_before.e2b_rep2_match_bus_file_fein_x != temp_before.e2b_rep2_match_bus_file_fein_y, 1, 0)
temp_before['e2b_rep2_busheader_ssn_flag'] = np.where(temp_before.e2b_rep2_busheader_ssn_x != temp_before.e2b_rep2_busheader_ssn_y, 1, 0)
temp_before['e2b_rep2_consheader_ssn_flag'] = np.where(temp_before.e2b_rep2_consheader_ssn_x != temp_before.e2b_rep2_consheader_ssn_y, 1, 0)
temp_before['e2b_rep2_idsearch_ssn_flag'] = np.where(temp_before.e2b_rep2_idsearch_ssn_x != temp_before.e2b_rep2_idsearch_ssn_y, 1, 0)
temp_before['e2b_rep2_ssn_on_file_flag'] = np.where(temp_before.e2b_rep2_ssn_on_file_x != temp_before.e2b_rep2_ssn_on_file_y, 1, 0)
temp_before['e2b_rep2_distance_addr_flag'] = np.where(temp_before.e2b_rep2_distance_addr_x != temp_before.e2b_rep2_distance_addr_y, 1, 0)
temp_before['out_rep3_addr_suffix_flag'] = np.where(temp_before.out_rep3_addr_suffix_x != temp_before.out_rep3_addr_suffix_y, 1, 0)
temp_before['bus2exec_rep3_desc_flag'] = np.where(temp_before.bus2exec_rep3_desc_x != temp_before.bus2exec_rep3_desc_y, 1, 0)
temp_before['e2b_rep3_on_inquiry_flag'] = np.where(temp_before.e2b_rep3_on_inquiry_x != temp_before.e2b_rep3_on_inquiry_y, 1, 0)
temp_before['e2b_rep3_match_bus_in_addr_flag'] = np.where(temp_before.e2b_rep3_match_bus_in_addr_x != temp_before.e2b_rep3_match_bus_in_addr_y, 1, 0)
temp_before['pop_rep3_ssn_flag'] = np.where(temp_before.pop_rep3_ssn_x != temp_before.pop_rep3_ssn_y, 1, 0)
temp_before['bus2exec_rep3_flag'] = np.where(temp_before.bus2exec_rep3_x != temp_before.bus2exec_rep3_y, 1, 0)
temp_before['e2b_rep3_name_on_file_flag'] = np.where(temp_before.e2b_rep3_name_on_file_x != temp_before.e2b_rep3_name_on_file_y, 1, 0)
temp_before['e2b_rep3_lexid_on_file_flag'] = np.where(temp_before.e2b_rep3_lexid_on_file_x != temp_before.e2b_rep3_lexid_on_file_y, 1, 0)
temp_before['e2b_rep3_on_property_cnt_flag'] = np.where(temp_before.e2b_rep3_on_property_cnt_x != temp_before.e2b_rep3_on_property_cnt_y, 1, 0)
temp_before['e2b_rep3_idsearch_name_flag'] = np.where(temp_before.e2b_rep3_idsearch_name_x != temp_before.e2b_rep3_idsearch_name_y, 1, 0)
temp_before['e2b_rep3_paw_match_flag'] = np.where(temp_before.e2b_rep3_paw_match_x != temp_before.e2b_rep3_paw_match_y, 1, 0)
temp_before['e2b_rep3_on_utility_flag'] = np.where(temp_before.e2b_rep3_on_utility_x != temp_before.e2b_rep3_on_utility_y, 1, 0)
temp_before['e2b_rep3_match_bus_file_first_flag'] = np.where(temp_before.e2b_rep3_match_bus_file_first_x != temp_before.e2b_rep3_match_bus_file_first_y, 1, 0)
temp_before['e2b_rep3_match_bus_file_last_flag'] = np.where(temp_before.e2b_rep3_match_bus_file_last_x != temp_before.e2b_rep3_match_bus_file_last_y, 1, 0)
temp_before['e2b_rep3_match_bus_file_full_flag'] = np.where(temp_before.e2b_rep3_match_bus_file_full_x != temp_before.e2b_rep3_match_bus_file_full_y, 1, 0)
temp_before['e2b_rep3_match_bus_file_addr_flag'] = np.where(temp_before.e2b_rep3_match_bus_file_addr_x != temp_before.e2b_rep3_match_bus_file_addr_y, 1, 0)
temp_before['e2b_rep3_idsearch_addr_flag'] = np.where(temp_before.e2b_rep3_idsearch_addr_x != temp_before.e2b_rep3_idsearch_addr_y, 1, 0)
temp_before['e2b_rep3_consheader_addr_flag'] = np.where(temp_before.e2b_rep3_consheader_addr_x != temp_before.e2b_rep3_consheader_addr_y, 1, 0)
temp_before['e2b_rep3_addr_on_file_flag'] = np.where(temp_before.e2b_rep3_addr_on_file_x != temp_before.e2b_rep3_addr_on_file_y, 1, 0)
temp_before['e2b_rep3_bus_addr_own_cnt_flag'] = np.where(temp_before.e2b_rep3_bus_addr_own_cnt_x != temp_before.e2b_rep3_bus_addr_own_cnt_y, 1, 0)
temp_before['e2b_rep3_distance_addr_flag'] = np.where(temp_before.e2b_rep3_distance_addr_x != temp_before.e2b_rep3_distance_addr_y, 1, 0)
temp_before['e2b_rep2_busheader_phone_flag'] = np.where(temp_before.e2b_rep2_busheader_phone_x != temp_before.e2b_rep2_busheader_phone_y, 1, 0)
temp_before['e2b_rep2_phn_on_file_flag'] = np.where(temp_before.e2b_rep2_phn_on_file_x != temp_before.e2b_rep2_phn_on_file_y, 1, 0)
temp_before['e2b_rep2_idsearch_phone_flag'] = np.where(temp_before.e2b_rep2_idsearch_phone_x != temp_before.e2b_rep2_idsearch_phone_y, 1, 0)
temp_before['e2b_rep2_match_bus_file_phn_flag'] = np.where(temp_before.e2b_rep2_match_bus_file_phn_x != temp_before.e2b_rep2_match_bus_file_phn_y, 1, 0)
temp_before['e2b_rep2_consheader_phone_flag'] = np.where(temp_before.e2b_rep2_consheader_phone_x != temp_before.e2b_rep2_consheader_phone_y, 1, 0)
temp_before['e2b_rep3_phn_on_file_flag'] = np.where(temp_before.e2b_rep3_phn_on_file_x != temp_before.e2b_rep3_phn_on_file_y, 1, 0)
temp_before['e2b_rep3_busheader_phone_flag'] = np.where(temp_before.e2b_rep3_busheader_phone_x != temp_before.e2b_rep3_busheader_phone_y, 1, 0)
temp_before['e2b_rep3_idsearch_phone_flag'] = np.where(temp_before.e2b_rep3_idsearch_phone_x != temp_before.e2b_rep3_idsearch_phone_y, 1, 0)
temp_before['e2b_rep3_match_bus_file_phn_flag'] = np.where(temp_before.e2b_rep3_match_bus_file_phn_x != temp_before.e2b_rep3_match_bus_file_phn_y, 1, 0)
temp_before['e2b_rep3_consheader_phone_flag'] = np.where(temp_before.e2b_rep3_consheader_phone_x != temp_before.e2b_rep3_consheader_phone_y, 1, 0)
temp_before['e2b_rep3_match_bus_file_fein_flag'] = np.where(temp_before.e2b_rep3_match_bus_file_fein_x != temp_before.e2b_rep3_match_bus_file_fein_y, 1, 0)
temp_before['e2b_rep3_idsearch_ssn_flag'] = np.where(temp_before.e2b_rep3_idsearch_ssn_x != temp_before.e2b_rep3_idsearch_ssn_y, 1, 0)
temp_before['e2b_rep3_consheader_ssn_flag'] = np.where(temp_before.e2b_rep3_consheader_ssn_x != temp_before.e2b_rep3_consheader_ssn_y, 1, 0)
temp_before['e2b_rep3_ssn_on_file_flag'] = np.where(temp_before.e2b_rep3_ssn_on_file_x != temp_before.e2b_rep3_ssn_on_file_y, 1, 0)
temp_before['e2b_rep3_busheader_ssn_flag'] = np.where(temp_before.e2b_rep3_busheader_ssn_x != temp_before.e2b_rep3_busheader_ssn_y, 1, 0)
temp_before['bus2exec_rep2_title_flag'] = np.where(temp_before.bus2exec_rep2_title_x != temp_before.bus2exec_rep2_title_y, 1, 0)
temp_before['bus2exec_rep3_title_flag'] = np.where(temp_before.bus2exec_rep3_title_x != temp_before.bus2exec_rep3_title_y, 1, 0)
temp_before['in_rep2_dob_flag'] = np.where(temp_before.in_rep2_dob_x != temp_before.in_rep2_dob_y, 1, 0)
temp_before['pop_rep2_dobyear_flag'] = np.where(temp_before.pop_rep2_dobyear_x != temp_before.pop_rep2_dobyear_y, 1, 0)
temp_before['pop_rep2_dob_flag'] = np.where(temp_before.pop_rep2_dob_x != temp_before.pop_rep2_dob_y, 1, 0)
temp_before['pop_rep2_dobday_flag'] = np.where(temp_before.pop_rep2_dobday_x != temp_before.pop_rep2_dobday_y, 1, 0)
temp_before['in_rep3_dob_flag'] = np.where(temp_before.in_rep3_dob_x != temp_before.in_rep3_dob_y, 1, 0)
temp_before['pop_rep3_dob_flag'] = np.where(temp_before.pop_rep3_dob_x != temp_before.pop_rep3_dob_y, 1, 0)
temp_before['pop_rep3_dobyear_flag'] = np.where(temp_before.pop_rep3_dobyear_x != temp_before.pop_rep3_dobyear_y, 1, 0)
temp_before['pop_rep3_dobday_flag'] = np.where(temp_before.pop_rep3_dobday_x != temp_before.pop_rep3_dobday_y, 1, 0)
temp_before['in_rep2_middle_flag'] = np.where(temp_before.in_rep2_middle_x != temp_before.in_rep2_middle_y, 1, 0)
temp_before['pop_rep2_middle_flag'] = np.where(temp_before.pop_rep2_middle_x != temp_before.pop_rep2_middle_y, 1, 0)
temp_before['in_rep3_middle_flag'] = np.where(temp_before.in_rep3_middle_x != temp_before.in_rep3_middle_y, 1, 0)
temp_before['pop_rep3_middle_flag'] = np.where(temp_before.pop_rep3_middle_x != temp_before.pop_rep3_middle_y, 1, 0)
temp_before['out_rep2_predir_flag'] = np.where(temp_before.out_rep2_predir_x != temp_before.out_rep2_predir_y, 1, 0)
temp_before['out_rep3_predir_flag'] = np.where(temp_before.out_rep3_predir_x != temp_before.out_rep3_predir_y, 1, 0)
temp_before['e2b_rep2_distance_phone_flag'] = np.where(temp_before.e2b_rep2_distance_phone_x != temp_before.e2b_rep2_distance_phone_y, 1, 0)
temp_before['e2b_rep3_distance_phone_flag'] = np.where(temp_before.e2b_rep3_distance_phone_x != temp_before.e2b_rep3_distance_phone_y, 1, 0)
temp_before['out_rep2_sec_range_flag'] = np.where(temp_before.out_rep2_sec_range_x != temp_before.out_rep2_sec_range_y, 1, 0)
temp_before['out_rep2_unit_desig_flag'] = np.where(temp_before.out_rep2_unit_desig_x != temp_before.out_rep2_unit_desig_y, 1, 0)
temp_before['out_rep3_sec_range_flag'] = np.where(temp_before.out_rep3_sec_range_x != temp_before.out_rep3_sec_range_y, 1, 0)
temp_before['out_rep3_unit_desig_flag'] = np.where(temp_before.out_rep3_unit_desig_x != temp_before.out_rep3_unit_desig_y, 1, 0)
temp_before['out_rep2_postdir_flag'] = np.where(temp_before.out_rep2_postdir_x != temp_before.out_rep2_postdir_y, 1, 0)
temp_before['pop_rep2_dobmonth_flag'] = np.where(temp_before.pop_rep2_dobmonth_x != temp_before.pop_rep2_dobmonth_y, 1, 0)
temp_before['out_rep3_postdir_flag'] = np.where(temp_before.out_rep3_postdir_x != temp_before.out_rep3_postdir_y, 1, 0)
temp_before['pop_rep3_dobmonth_flag'] = np.where(temp_before.pop_rep3_dobmonth_x != temp_before.pop_rep3_dobmonth_y, 1, 0)
temp_before['e2b_rep1_consheader_addr_flag'] = np.where(temp_before.e2b_rep1_consheader_addr_x != temp_before.e2b_rep1_consheader_addr_y, 1, 0)
temp_before['in_rep2_dlnumber_flag'] = np.where(temp_before.in_rep2_dlnumber_x != temp_before.in_rep2_dlnumber_y, 1, 0)
temp_before['pop_rep2_dlnumber_flag'] = np.where(temp_before.pop_rep2_dlnumber_x != temp_before.pop_rep2_dlnumber_y, 1, 0)
temp_before['pop_rep3_dlnumber_flag'] = np.where(temp_before.pop_rep3_dlnumber_x != temp_before.pop_rep3_dlnumber_y, 1, 0)
temp_before['in_rep2_dlstate_flag'] = np.where(temp_before.in_rep2_dlstate_x != temp_before.in_rep2_dlstate_y, 1, 0)
temp_before['pop_rep2_dlstate_flag'] = np.where(temp_before.pop_rep2_dlstate_x != temp_before.pop_rep2_dlstate_y, 1, 0)
temp_before['in_rep3_dlnumber_flag'] = np.where(temp_before.in_rep3_dlnumber_x != temp_before.in_rep3_dlnumber_y, 1, 0)
temp_before['pop_rep3_dlstate_flag'] = np.where(temp_before.pop_rep3_dlstate_x != temp_before.pop_rep3_dlstate_y, 1, 0)
temp_before['in_rep3_dlstate_flag'] = np.where(temp_before.in_rep3_dlstate_x != temp_before.in_rep3_dlstate_y, 1, 0)
temp_before['in_rep3_email_flag'] = np.where(temp_before.in_rep3_email_x != temp_before.in_rep3_email_y, 1, 0)
temp_before['in_rep2_email_flag'] = np.where(temp_before.in_rep2_email_x != temp_before.in_rep2_email_y, 1, 0)
temp_before['pop_rep3_age_flag'] = np.where(temp_before.pop_rep3_age_x != temp_before.pop_rep3_age_y, 1, 0)
temp_before['pop_rep2_age_flag'] = np.where(temp_before.pop_rep2_age_x != temp_before.pop_rep2_age_y, 1, 0)
temp_before['in_rep3_age_flag'] = np.where(temp_before.in_rep3_age_x != temp_before.in_rep3_age_y, 1, 0)
temp_before['in_rep2_age_flag'] = np.where(temp_before.in_rep2_age_x != temp_before.in_rep2_age_y, 1, 0)

/tmp/ipykernel_31941/3756914514.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_before['in_rep2_first_flag'] = np.where(temp_before.in_rep2_first_x != temp_before.in_rep2_first_y, 1, 0)
/tmp/ipykernel_31941/3756914514.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp_before['pop_rep2_last_flag'] = np.where(temp_before.pop_rep2_last_x != temp_before.pop_rep2_last_y, 1, 0)
/tmp/ipykernel_31941/3756914514.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

In [14]:
field = ["account", 'in_rep2_first',
'pop_rep2_last',
'pop_rep2_first',
'e2b_rep2_match_bus_in_first',
'e2b_rep2_match_bus_in_last',
'e2b_rep2_match_bus_in_full',
'out_rep2_addr_status',
'pop_rep2_zip',
'pop_rep2_addr',
'in_rep2_streetaddress1',
'in_rep2_zip',
'in_rep2_city',
'pop_rep2_city',
'out_rep2_state',
'out_rep2_addr_type',
'out_rep2_prim_name',
'out_rep2_zip5',
'out_rep2_lat',
'out_rep2_long',
'out_rep2_geo_block',
'out_rep2_county',
'out_rep2_prim_range',
'pop_rep2_ssn',
'lexid_rep2',
'pop_rep2_state',
'in_rep2_state',
'out_rep2_zip4',
'out_rep2_addr_suffix',
'in_rep2_phone10',
'pop_rep2_phone',
'e2b_rep2_on_inquiry',
'bus2exec_rep2_desc',
'e2b_rep2_match_bus_in_addr',
'in_rep3_first',
'e2b_rep3_match_bus_in_last',
'pop_rep3_first',
'pop_rep3_last',
'e2b_rep3_match_bus_in_first',
'e2b_rep3_match_bus_in_full',
'out_rep3_addr_status',
'in_rep3_streetaddress1',
'pop_rep3_addr',
'in_rep3_zip',
'pop_rep3_zip',
'in_rep3_city',
'pop_rep3_city',
'pop_rep3_state',
'in_rep3_state',
'out_rep3_state',
'out_rep3_prim_name',
'out_rep3_addr_type',
'out_rep3_zip5',
'out_rep3_geo_block',
'out_rep3_long',
'out_rep3_lat',
'out_rep3_county',
'out_rep3_prim_range',
'lexid_rep3',
'in_rep3_phone10',
'pop_rep3_phone',
'bus2exec_rep2',
'out_rep3_zip4',
'e2b_rep2_idsearch_name',
'e2b_rep2_lexid_on_file',
'e2b_rep2_name_on_file',
'e2b_rep2_on_property_cnt',
'e2b_rep2_on_utility',
'e2b_rep2_match_bus_file_full',
'e2b_rep2_paw_match',
'e2b_rep2_match_bus_file_first',
'e2b_rep2_match_bus_file_last',
'e2b_rep2_idsearch_addr',
'e2b_rep2_match_bus_file_addr',
'e2b_rep2_consheader_addr',
'e2b_rep2_addr_on_file',
'e2b_rep2_bus_addr_own_cnt',
'e2b_rep2_match_bus_file_fein',
'e2b_rep2_busheader_ssn',
'e2b_rep2_consheader_ssn',
'e2b_rep2_idsearch_ssn',
'e2b_rep2_ssn_on_file',
'e2b_rep2_distance_addr',
'out_rep3_addr_suffix',
'bus2exec_rep3_desc',
'e2b_rep3_on_inquiry',
'e2b_rep3_match_bus_in_addr',
'pop_rep3_ssn',
'bus2exec_rep3',
'e2b_rep3_name_on_file',
'e2b_rep3_lexid_on_file',
'e2b_rep3_on_property_cnt',
'e2b_rep3_idsearch_name',
'e2b_rep3_paw_match',
'e2b_rep3_on_utility',
'e2b_rep3_match_bus_file_first',
'e2b_rep3_match_bus_file_last',
'e2b_rep3_match_bus_file_full',
'e2b_rep3_match_bus_file_addr',
'e2b_rep3_idsearch_addr',
'e2b_rep3_consheader_addr',
'e2b_rep3_addr_on_file',
'e2b_rep3_bus_addr_own_cnt',
'e2b_rep3_distance_addr',
'e2b_rep2_busheader_phone',
'e2b_rep2_phn_on_file',
'e2b_rep2_idsearch_phone',
'e2b_rep2_match_bus_file_phn',
'e2b_rep2_consheader_phone',
'e2b_rep3_phn_on_file',
'e2b_rep3_busheader_phone',
'e2b_rep3_idsearch_phone',
'e2b_rep3_match_bus_file_phn',
'e2b_rep3_consheader_phone',
'e2b_rep3_match_bus_file_fein',
'e2b_rep3_idsearch_ssn',
'e2b_rep3_consheader_ssn',
'e2b_rep3_ssn_on_file',
'e2b_rep3_busheader_ssn',
'bus2exec_rep2_title',
'bus2exec_rep3_title',
'in_rep2_dob',
'pop_rep2_dobyear',
'pop_rep2_dob',
'pop_rep2_dobday',
'in_rep3_dob',
'pop_rep3_dob',
'pop_rep3_dobyear',
'pop_rep3_dobday',
'in_rep2_middle',
'pop_rep2_middle',
'in_rep3_middle',
'pop_rep3_middle',
'out_rep2_predir',
'out_rep3_predir',
'e2b_rep2_distance_phone',
'e2b_rep3_distance_phone',
'out_rep2_sec_range',
'out_rep2_unit_desig',
'out_rep3_sec_range',
'out_rep3_unit_desig',
'out_rep2_postdir',
'pop_rep2_dobmonth',
'out_rep3_postdir',
'pop_rep3_dobmonth',
'e2b_rep1_consheader_addr',
'in_rep2_dlnumber',
'pop_rep2_dlnumber',
'pop_rep3_dlnumber',
'in_rep2_dlstate',
'pop_rep2_dlstate',
'in_rep3_dlnumber',
'pop_rep3_dlstate',
'in_rep3_dlstate',
'in_rep3_email',
'in_rep2_email',
'pop_rep3_age',
'pop_rep2_age',
'in_rep3_age',
'in_rep2_age',
]